# Processing Images For Merged Dataset

Here we enhance the images (training and testing) to improve the image quality using usuyama style ben graham

Define display images function

In [1]:
import os
import cv2
from matplotlib import pyplot as plt
def display_images(image_filenames:list[str], title="Image DataSet", col_count = 10):
    plt.suptitle(title, fontsize=20)
    img_count = len(image_filenames)
    plt.figure(figsize=(15, 15))
    for i in range(1, img_count+1):
        plt.subplot(5, col_count, i)
        image_path=image_filenames[i-1]
        image_name=os.path.basename(image_path)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img, aspect="auto")
        plt.title(f"{image_name}")
        plt.axis(False)
    plt.show();

Define our image processor class

In [2]:
import cv2
import numpy as np
from pathlib import Path

class ODIRImageProcessor:
    """Simple processor for ODIR retinal images"""
    
    def __init__(self, output_size=512):
        self.output_size = output_size
    
    def process(self, input_path, output_path=None, skip_if_exist=True):
        """
        Process a single ODIR image
        
        Args:
            input_path: Path to input image
            output_path: Optional path to save result. If None, returns image array
        
        Returns:
            Processed image as numpy array
        """
        if skip_if_exist and output_path is not None:
            if os.path.exists(output_path):
                print(f"Skip as processed image existed: {output_path}")
                return

        # Read image
        img = cv2.imread(str(input_path))
        if img is None:
            raise ValueError(f"Cannot read image: {input_path}")
        
        # Convert BGR to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Call Preprocess script
        img = self.usuyama_prep(img=img)

        # Convert back to BGR for saving/display
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        # Save if output path is provided
        if output_path:
            cv2.imwrite(str(output_path), img_bgr)
            print(f"Saved processed image to: {output_path}")
        
       # return img_bgr

    def resize_odir_image(self, img, target_size=512):
        """
        Combines Circular Cropping, Aspect-Ratio Resizing, and Padding.
        """
        # 1. Load and initial crop to remove obvious black dead space
        if img is None: return None
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Thresholding to find the retina boundary
        _, mask = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
        coords = cv2.findNonZero(mask)
        
        if coords is not None:
            x, y, w, h = cv2.boundingRect(coords)
            img = img[y:y+h, x:x+w]

        # 2. Letterbox Resize (Preserve Aspect Ratio)
        h, w = img.shape[:2]
        scale = target_size / max(h, w)
        new_w, new_h = int(w * scale), int(h * scale)
        
        # Use INTER_AREA for high-quality downsampling
        resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        
        # 3. Create Square Canvas and Center
        final_img = np.zeros((target_size, target_size, 3), dtype=np.uint8)
        offset_y = (target_size - new_h) // 2
        offset_x = (target_size - new_w) // 2
        final_img[offset_y:offset_y+new_h, offset_x:offset_x+new_w] = resized
        
        return final_img
    def usuyama_prep(self, img):
        """Enhances vessels and normalizes lighting."""
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Circular Crop: Find non-black pixels and crop
        img = self.resize_odir_image(img, target_size=self.output_size)  # Crop to circular region
        blurred = cv2.GaussianBlur(img, (0, 0), 10)
        enhanced = cv2.addWeighted(img, 4, blurred, -4, 128)
        return enhanced

    def remove_outer_circle(self, a, p=0.9, r=256):
        b = np.zeros(a.shape, dtype=np.uint8)
        cv2.circle(b, (a.shape[1] // 2, a.shape[0] // 2), int(r * p), (1, 1, 1), -1, 8, 0)
        
        return a * b + 128 * (1 - b)
    
    def place_in_square(self, img, r, h=512, w=512):
        new_img = np.zeros((2 * r, 2 * r, 3), dtype=np.uint8)
        new_img += 128
        new_img[r - h // 2:r - h // 2 + img.shape[0], r - w // 2:r - w // 2 + img.shape[1]] = img
        
        return new_img
    def process_folder(self, input_folder, output_folder, skip_if_exist, extension=".jpg"):
        """
        Process all images in a folder
        """
        input_folder = Path(input_folder)
        output_folder = Path(output_folder)
        output_folder.mkdir(parents=True, exist_ok=True)
        
        image_files = list(input_folder.glob(f"*{extension}"))
        print(f"Found {len(image_files)} images to process")
        
        for img_path in image_files:
            output_path = output_folder / img_path.name
            try:
                self.process(img_path, output_path, skip_if_exist=skip_if_exist)
            except Exception as e:
                print(f"Error processing {img_path.name}: {e}")


Test on 1 image and then dispaly and compare

In [ ]:
!mkdir -p ODIR-2019/YOLO/processed_512u_merged

In [ ]:
import os
input_folder="ODIR-2019/YOLO/preprocessed_merged"
output_folder="ODIR-2019/YOLO/processed_512u_merged"
processor = ODIRImageProcessor(output_size=512)

for root, dirs, files in os.walk(input_folder):
    for dir_name in dirs:
        source_dir=os.path.join(root, dir_name)
        target_dir=source_dir.replace(input_folder, output_folder)
        os.makedirs(target_dir, exist_ok=True)
        processor.process_folder(
            input_folder=source_dir,
            output_folder=target_dir,
            skip_if_exist=True,
            extension=".jpg"
        )
        processor.process_folder(
            input_folder=source_dir,
            output_folder=target_dir,
            skip_if_exist=True,
            extension=".png"
        )


Found 0 images to process
Found 0 images to process
Found 0 images to process
Found 0 images to process
Found 0 images to process
Found 0 images to process
Found 26 images to process
Saved processed image to: ODIR-2019/YOLO/processed_512ua_merged/val/cataract/2108_left.jpg
Saved processed image to: ODIR-2019/YOLO/processed_512ua_merged/val/cataract/294_left.jpg
Saved processed image to: ODIR-2019/YOLO/processed_512ua_merged/val/cataract/0_left.jpg
Saved processed image to: ODIR-2019/YOLO/processed_512ua_merged/val/cataract/2150_left.jpg
Saved processed image to: ODIR-2019/YOLO/processed_512ua_merged/val/cataract/2159_left.jpg
Saved processed image to: ODIR-2019/YOLO/processed_512ua_merged/val/cataract/1435_left.jpg
Saved processed image to: ODIR-2019/YOLO/processed_512ua_merged/val/cataract/330_left.jpg
Saved processed image to: ODIR-2019/YOLO/processed_512ua_merged/val/cataract/2243_left.jpg
Saved processed image to: ODIR-2019/YOLO/processed_512ua_merged/val/cataract/2130_right.jpg
Sa

KeyboardInterrupt: 